In [12]:
import pdb
import glob
import json
import sys
import smart_open
from datetime import date
from smart_open import s3_iter_bucket

In [2]:
from impresso_commons.utils.s3 import get_s3_connection
from impresso_commons.utils.s3 import s3_get_articles, s3_get_pages
from impresso_commons.path import IssueDir, detect_issues
from impresso_commons.path import s3_detect_issues

In [3]:
from impresso_commons.text.rebuilder import *

In [7]:
basedir = "/Users/rromanello/Documents/impresso/impresso-text-acquisition/text_importer/data/out/"

In [8]:
issues = detect_issues(basedir)

In [11]:
issues[0]

IssueDirectory(journal='GDL', date=datetime.date(1900, 1, 2), edition='a', path='/Users/rromanello/Documents/impresso/impresso-text-acquisition/text_importer/data/out/GDL/1900/01/02')

In [32]:
def read_pages(issue):
    issue_file = next(iter(glob.glob(f"{issue.path}/a/*issue.json")))
    
    with open(issue_file, "r") as ifile:
        data = json.load(ifile)
    
    pages = []
    page_files = data["pp"]
    for file in page_files:
        with open(os.path.join(issue.path, 'a', f"{file}.json"), 'r') as ifile:
            pages.append(json.load(ifile))
    
    return pages

In [89]:
def rebuild_text(tokens, string=None):
    """The text rebuilding function."""

    def get_space(token):
        return " "

    regions = []

    if string is None:
        string = ""

    for token in tokens:
        region = {}
        region["coords"] = token["c"]
        region["start"] = len(string)
        region["surface"] = None

            
        if "hy" in token:
            region["length"] = len(token["tx"][:-1])
            region["surface"] = token["tx"][:-1]
            
        elif "nf" in token:
            region["length"] = len(token["nf"])
            
            if "gn" in token and token["gn"]:
                tmp = "{}".format(token["nf"])
                region["surface"] = token["tx"]
                string += tmp
            else:
                tmp = "{} ".format(token["nf"])
                region["surface"] = token["tx"]
                string += tmp
        else:
            region["length"] = len(token["tx"])
            
            if "gn" in token and token["gn"]:
                tmp = "{}".format(token["tx"])
                region["surface"] = tmp
                string += tmp
            else:
                tmp = "{} ".format(token["tx"])
                region["surface"] = tmp
                string += tmp
        
        regions.append(region)
            

    return (string, regions)

In [90]:
def rebuild_page(page_obj):
    fulltext = ""
    regions = page_obj["r"]
    tokens = [
            token
            for region in regions
            for para in region["p"]
            for line in para["l"]
            for token in line["t"]
        ]

    fulltext, regions = rebuild_text(tokens)
    return (fulltext, regions)

In [91]:
pp = read_pages(issues[0])
for p in pp[:1]:
    text, regions = rebuild_page(p)
    print(text)

RÉDACTION Bat Pépins i BUREAU D'ABONNEMENTS Ruelle Si-François, 20-Oo s'abouno dans tous les bureaux de posta Les abonnements partent du i" ou du 15 de chaque m ». * Gazelle de Lausanne ET JOURNAL SUISSE FONDÉE EN 1798 PRIX D'ABONNEMENl Ul U (BtÛ I Mit Strisse Fr. 20 10 50 5 50 Union postale » 36 18 50 9 50 Prix du numéro : 10 centimes. Lausanne, 2 janvier 1900. Regards en arrière. Supposez un dessinateur installé au coin de son feu tandis que la dernière bûche de l'année s'effondrait bleuâtre entre les chenets. Supposez-le cherchant à personnifier le peuple suisse, comme d'au 1 res ont évoqué en « oncle Sam » les Yankees, ou, en John Bnll, la nation britannique. La silhouette découlant de la pointe de son crayon aurait été environ la suivante : Un gaillard carré d'épaules, visage placide et énergique, léger embonpoint, mains robustes de travailleur, solidement assis et non moins solidement vêtu. Semelles puissantes. Sur la table, un verre. Plus loin, probablement, un journal. Très cer

In [92]:
for region in regions:
    start = region["start"]
    end = start + region["length"]
    print(region["surface"], text[start:end])

RÉDACTION  RÉDACTION
Bat  Bat
Pépins  Pépins
i  i
BUREAU  BUREAU
D'ABONNEMENTS  D'ABONNEMENTS
Ruelle  Ruelle
Si-François Si-François
,  ,
20 20
Oo 20-Oo
s'abouno  s'abouno
dans  dans
tous  tous
les  les
bureaux  bureaux
de  de
posta  posta
Les  Les
abonnements  abonnements
partent  partent
du  du
i"  i"
ou  ou
du  du
15  15
de  de
chaque  chaque
m  m
» »
.  .
*  *
Gazelle  Gazelle
de  de
Lausanne  Lausanne
ET  ET
JOURNAL  JOURNAL
SUISSE  SUISSE
FONDÉE  FONDÉE
EN  EN
1798  1798
PRIX  PRIX
D'ABONNEMENl  D'ABONNEMENl
Ul  Ul
U  U
( (
BtÛ  BtÛ
I  I
Mit  Mit
Strisse  Strisse
Fr Fr
.  .
20  20
10  10
50  50
5  5
50  50
Union  Union
postale  postale
»  »
36  36
18  18
50  50
9  9
50  50
Prix  Prix
du  du
numéro  numéro
:  :
10  10
centimes centimes
.  .
Lausanne Lausanne
,  ,
2  2
janvier  janvier
1900 1900
.  .
Regards  Regards
en  en
arrière arrière
.  .
Supposez  Supposez
un  un
dessinateur  dessinateur
installé  installé
au  au
coin  coin
de  de
son  son
feu  feu
tandis  tandis
que  que
la

la  la
douce  douce
pluie  pluie
d'or d'or
,  ,
une  une
plante  plante
lourde lourde
,  ,
maussade  maussade
et  et
peu  peu
élégante élégante
,  ,
la  la
bureaucratie  bureaucratie
fédérale fédérale
,  ,
pous pous
sera poussera
encore  encore
quelques  quelques
nouvelles  nouvelles
racines  racines
dans  dans
le  le
bon  bon
terroir  terroir
d'Helvélie d'Helvélie
.  .
Avant  Avant
d'adopter  d'adopter
la  la
loi  loi
d'assurance d'assurance
,  ,
nos  nos
députés  députés
se  se
sont  sont
livrés  livrés
à  à
quelques  quelques
petits  petits
jeux  jeux
d'équilibristes d'équilibristes
.  .
Ces  Ces
exercices exercices
,  ,
en  en
général  général
innoffensifs  innoffensifs
pour  pour
la  la
galerie galerie
,  ,
le  le
sont  sont
peut-être  peut-être
moins  moins
lorsqu'ils  lorsqu'ils
ont  ont
lieu  lieu
sur  sur
le  le
terrain  terrain
du  du
budget budget
.  .
L'équilibre  L'équilibre
financier  financier
auquel  auquel
ont  ont
abouti  abouti
les  les
débats  débats
est  est
d'aill

se se
ront seront
,  ,
dans  dans
l'idée  l'idée
du  du
pouvoir  pouvoir
fédéral fédéral
,  ,
cons cons
tituées constituées
dès  dès
l'été  l'été
prochain prochain
.  .
Elles  Elles
auront  auront
ainsi  ainsi
plus  plus
de  de
deux  deux
ans  ans
devant  devant
elles  elles
pour  pour
préparer  préparer
la  la
reprise  reprise
du  du
réseau  réseau
au  au
1  1
jan jan
vier janvier
1903 1903
.  .
Une  Une
loi  loi
sur  sur
les  les
tarifs  tarifs
des  des
che che
mins chemins
de  de
fer  fer
fédéraux  fédéraux
a  a
été  été
soumise  soumise
aux  aux
Chambres Chambres
.  .
Celles-ci  Celles-ci
viennent  viennent
de  de
voter  voter
la  la
loi  loi
sur  sur
les  les
chemins  chemins
de  de
fer  fer
secondaires secondaires
.  .
Quant  Quant
à  à
la  la
loi  loi
sur  sur
la  la
durée  durée
du  du
travail  travail
dans  dans
les  les
entreprises  entreprises
de  de
transport transport
,  ,
le  le
Conseil  Conseil
des  des
Etats  Etats
n'en  n'en
a  a
pas  pas
abordé  abordé
la  la
discussi

à  à
ne  ne
pas  pas
douter  douter
de  de
mon  mon
désintéresse désintéresse
ment désintéressement
et  et
à  à
trouver  trouver
que  que
ma  ma
présence  présence
auprès  auprès
d'elle d'elle
,  ,
en  en
de  de
pareilles  pareilles
circonstances circonstances
,  ,
constitue  constitue
la  la
plus  plus
grande  grande
preuve  preuve
d'affection  d'affection
qu'un  qu'un
homme  homme
puisse  puisse
donner  donner
:  :
le  le
sacrifice  sacrifice
d'une  d'une
partie  partie
de  de
son  son
honneur honneur
.  .
S'arrêtant  S'arrêtant
brusquement brusquement
,  ,
le  le
prince  prince
secoua  secoua
ses  ses
larges  larges
épaules  épaules
d'un  d'un
mouvement  mouvement
symbolique symbolique
,  ,
comme  comme
s  s
il  il
rechargeait  rechargeait
le  le
poids  poids
du  du
silence  silence
un  un
instant  instant
déposé déposé
.  .
Puis  Puis
il  il
tourna  tourna
sur  sur
ses  ses
talons talons
,  ,
s'en  s'en
alla  alla
de  de
son  son
pas  pas
raide raide
,  ,
repassa  repassa
indiffère

souhaiter  souhaiter
de  de
vivre vivre
,  ,
la  la
pauvre  pauvre
femme  femme
I  I
Tout  Tout
ce  ce
que  que
voas  voas
vous  vous
voudrez voudrez
...  ...
mais  mais
on  on
n'en  n'en
a  a
pas  pas
moins  moins
de  de
la  la
peine peine
,  ,
je  je
vous  vous
assure assure
...  ...
beau beau
coup beaucoup
de  de
peine peine
...  ...
On  On
pourrait  pourrait
dire  dire
mieux  mieux
;  ;
on  on
ne  ne
peut  peut
guère  guère
dire  dire
plus plus
.  .
Mlle  Mlle
de  de
Moatferrand  Moatferrand
appartient  appartient
évi évi
demment évidemment
à  à
la  la
catégorie  catégorie
des  des
morts  morts
qui  qui
ne  ne
font  font
pas  pas
de  de
vide vide
,  ,
qui  qui
font  font
de  de
la  la
place place
,  ,
et  et
le  le
chagrin  chagrin
île  île
la  la
perdre  perdre
ne  ne
contre-balance  contre-balance
pas  pas
l'agrément  l'agrément
de  de
recueillir  recueillir
sa  sa
fortune fortune
.  .
Ses  Ses
héritiers  héritiers
se  se
sont  sont
montrés  montrés
consciencieux  consciencieux
e

,  ,
le  le
royaume  royaume
des  des
Deux-Sicile  Deux-Sicile
? ?
,  ,
les  les
États  États
pontificaux pontificaux
... ...
)  )
et  et
plus  plus
rarement  rarement
encore encore
,  ,
y  y
forent  forent
mis  mis
à  à
même  même
de  de
batailler  batailler
pour  pour
de  de
bon bon
.  .
C'est  C'est
alors  alors
que  que
commença  commença
pour  pour
ces  ces
militai militai
res militaires
suisses suisses
,  ,
et  et
plus  plus
spécialement  spécialement
pour  pour
leurs  leurs
di di
gnes dignes
officiers officiers
,  ,
une  une
période  période
de  de
vide  vide
et  et
d'ennui d'ennui
,  ,
d'autant  d'autant
plus  plus
pesante  pesante
et  et
énervante  énervante
que que
,  ,
en  en
vertu  vertu
d'un  d'un
long  long
atavisme atavisme
,  ,
bouillonnait  bouillonnait
toujours  toujours
dans  dans
leurs  leurs
veines  veines
ce  ce
sang  sang
chaud  chaud
et  et
généreux  généreux
dont  dont
on  on
ne  ne
les  les
vit  vit
jamais  jamais
avares avares
.  .
Non  Non
moins  moins
taill